# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096959


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:02,  2.23s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:51,  1.98s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:34,  1.42s/it]

Rendering models:  27%|██▋       | 8/30 [00:07<00:22,  1.02s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:01,  4.97it/s]

Rendering models:  83%|████████▎ | 25/30 [00:09<00:00,  6.12it/s]

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  7.13it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  7.99it/s]

not-logged-in-3b0b929f4475fd276561    0.000357
Jnursssmith5263                       0.000177
Quarkgc                               0.000333
deivad                                0.000208
Galactea                              0.000388
wilderbeast2                          0.499107
boomod                                0.000266
awright5                              0.000378
salmi.a01                             0.000419
not-logged-in-b54d389b0f330740e262    0.000349
not-logged-in-9a45911eb71184fe7c7f    0.000193
not-logged-in-5f30b2bb9b7f9d49da41    0.986039
not-logged-in-09330e0b15263888ee9d    0.000438
crush202020                           0.000268
quirito19                             0.000244
jayton                                0.000193
pangeli5                              0.000272
peakscience7                          0.000190
clairedeu                             0.000213
mjcb4                                 0.002348
rhondacard                            0.001706
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())